
## load data

In [1]:
import sys
import pandas as pd
sys.path.append('/Users/shinbo/PycharmProjects/paper/LDA')

def print_top_words(lam, feature_names, n_top_words):
    for topic_id, topic in enumerate(lam):
        print('\nTopic Nr.%d:' % int(topic_id + 1))
        print(''.join([feature_names[i] + ' ' + str(round(topic[i], 2))
                       + ' | ' for i in topic.argsort()[:-n_top_words - 1:-1]]))
        
def TOP_N_WORDS_df(MODEL, TOP_N_WORDS, col_names):
    DMM_LDA_Top_words = pd.DataFrame()
    for i in range(4):
        temp = pd.DataFrame({'words':cv.get_feature_names(), 'lambda':MODEL.components_[i,:]})
        temp = temp.sort_values(by='lambda', ascending=False).iloc[:TOP_N_WORDS,:]
        DMM_LDA_Top_words[i] = temp['words'].tolist()
    DMM_LDA_Top_words.columns = col_names
    return DMM_LDA_Top_words

In [7]:
import pickle
from sklearn.feature_extraction.text import CountVectorizer

dir_ = '/Users/shinbo/Desktop/metting/LDA/0. data/JH_data/JH_data_cleaned.pkl'

data = pickle.load(open(dir_, 'rb'))
stop_words = ['good','great','go','get','place']
for i in range(len(data)):
    data[i] = [w for w in data[i] if w not in stop_words]

data_join = [' '.join(doc) for doc in data]
cv = CountVectorizer()
X = cv.fit_transform(data_join).toarray()

## DMM-LDA Result

In [9]:
dir_model = f'/Users/shinbo/Desktop/metting/LDA/0. data/JH_data/result_4_topics.pkl'
DMM_LDA = pickle.load(open(dir_model,'rb'))
TOP_N_WORDS = 100
DMM_LDA_Top_words = TOP_N_WORDS_df(DMM_LDA, TOP_N_WORDS, [f'Topic {i}' for i in range(1,5)])

DMM_LDA_lam =  [DMM_LDA.components_[k,:] for k in range(4)]
print_top_words(DMM_LDA_lam, list(cv.get_feature_names()), 20)


Topic Nr.1:
food 2458.24 | restaurant 1322.13 | service 1033.32 | price 768.9 | time 645.99 | try 633.13 | dish 610.3 | order 576.53 | make 569.66 | one 563.26 | meal 559.97 | come 542.81 | like 542.2 | menu 541.59 | eat 518.95 | lunch 481.02 | well 480.0 | fresh 463.18 | staff 460.67 | friendly 456.68 | 

Topic Nr.2:
food 851.39 | order 658.84 | come 455.5 | time 394.55 | restaurant 377.43 | eat 350.38 | wait 344.93 | one 336.59 | table 327.83 | service 325.78 | take 324.96 | like 313.82 | back 307.02 | make 286.48 | menu 284.66 | well 236.98 | meal 228.13 | breakfast 217.76 | say 213.58 | look 212.93 | 

Topic Nr.3:
food 1032.58 | service 499.35 | restaurant 429.52 | time 347.06 | one 321.48 | make 317.5 | well 289.85 | find 269.27 | best 267.86 | price 252.77 | dinner 246.43 | staff 235.08 | come 229.96 | delicious 225.85 | try 220.77 | back 218.66 | like 216.83 | enjoy 214.27 | menu 204.92 | love 204.63 | 

Topic Nr.4:
food 887.19 | order 616.05 | like 555.55 | chicken 537.41 | on

In [10]:
TOP_N_WORDS = 100
DMM_LDA_Top_words = TOP_N_WORDS_df(DMM_LDA, TOP_N_WORDS, [f'Topic {i}' for i in range(1,5)])

In [31]:
DMM_LDA.perplexity

[6151.6169339328235,
 5364.355225616776,
 5362.65730105442,
 5361.525234035369,
 5361.4885118276825,
 5361.488380038819]

## DMM-CLDA Result

In [12]:
dir_model = '/Users/shinbo/PycharmProjects/model_lda/HOL/CDMM_result_800_stop_words_holulu.pkl'
DMM_CLDA = pickle.load(open(dir_model,'rb'))
DMM_CLDA_lam =  [DMM_CLDA.components_[k,:] for k in range(4)]
TOP_N_WORDS = 100
DMM_CLDA_Top_words = TOP_N_WORDS_df(DMM_CLDA, TOP_N_WORDS, [f'Topic {i}' for i in range(1,5)])
DMM_CLDA_Top_words.iloc[:20,]

,Topic 1,Topic 2,Topic 3,Topic 4
0,food,service,food,food
1,price,serve,restaurant,location
2,restaurant,staff,dinner,stay
3,service,friendly,coffee,close
4,value,check,drink,front
5,pay,offer,breakfast,far
6,cheap,parking,lunch,bathroom
7,expensive,manager,tea,order
8,time,helpful,water,balcony
9,reasonably,help,cafe,lobby


In [32]:
DMM_CLDA.perplexity

[9583.259897289177,
 7860.4060265047665,
 7825.5108801118595,
 7803.606333616812,
 7797.880270478682,
 7795.743914528182,
 7791.14747563514,
 7789.870789021993,
 7789.643356681828,
 7789.643354936834]

In [13]:
import pandas as pd
import numpy as np
from collections import Counter
def top_20_words_for_topics(CLDA_lam,CDMM, aux=None, n=50):
    if aux is True:
        K = len(CLDA_lam)-1
    else:
        K = len(CLDA_lam)
    words = {}
    unique_words = []
    for k in range(K):
        CLDA_lam_word = pd.DataFrame({'word':cv.get_feature_names(), 'lam':CDMM.components_[k,:] }).sort_values(by='lam', ascending=False)
        top_words = CLDA_lam_word.iloc[:n,:]['word'].tolist()
        words[k] = CLDA_lam_word.iloc[:n,:].set_index('word').to_dict()['lam']
        unique_words += top_words
        
    c = Counter(unique_words)
    words_all_appear = []
    for k,v in c.items():
        if v == K:
            words_all_appear.append(k)
    
    
    
    words_degree = []
    for k in range(K):
        for w,d in words[k].items():
            if w in words_all_appear:
                words_degree.append(d)
        
    return words_all_appear, len(set(unique_words)) / (K*n), len(set(unique_words)) , np.mean(words_degree)
words_, uq_words_ratio, uq_words_nums, asso_deg = top_20_words_for_topics(DMM_LDA_lam,DMM_LDA, n=50)
uq_words_ratio, uq_words_nums

(0.39, 78)

In [14]:
words_, uq_words_ratio, uq_words_nums, asso_deg = top_20_words_for_topics(DMM_CLDA_lam,DMM_CLDA, n=50)
uq_words_ratio, uq_words_nums

(0.57, 114)

## n 변화시켜가며 비율 살펴보기

In [15]:
def varying_n(lam,mod):
    result = {}
    for n in range(10,51,10):
        words_, uq_words_ratio, uq_words_nums, asso_deg = top_20_words_for_topics(lam, mod, n=n)
        result[n] = (uq_words_ratio, uq_words_nums)
    return result

In [16]:
varying_n(DMM_LDA_lam,DMM_LDA)

{10: (0.5, 20),
 20: (0.4625, 37),
 30: (0.45, 54),
 40: (0.4125, 66),
 50: (0.39, 78)}

In [17]:
varying_n(DMM_CLDA_lam,DMM_CLDA)

/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3335: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/shinbo/opt/anaconda3/lib/python3.7/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


{10: (0.9, 36),
 20: (0.8375, 67),
 30: (0.6833333333333333, 82),
 40: (0.61875, 99),
 50: (0.57, 114)}

## Glove 벡터 이용

In [18]:
n=0
glove_dir = '/Users/shinbo/Desktop/metting/LDA/paper/word_embedding/glove.6B/glove.6B.100d.txt'
word2glove = {}
with open(glove_dir, encoding="utf8") as f:

    for line in f:
        word_vector = line.split() # 각 줄을 읽어와서 word_vector에 저장.
        word = word_vector[0] # word_vector에서 첫번째 값만 저장
        word2glove[word] = np.array([float(i) for i in word_vector[1:]])

In [20]:
from scipy.spatial.distance import cosine
TOP_N_WORDS = 10
DMM_LDA_Top_words = TOP_N_WORDS_df(DMM_LDA, TOP_N_WORDS, [f'Topic {i}' for i in range(1,5)])
DMM_CLDA_Top_words = TOP_N_WORDS_df(DMM_CLDA, TOP_N_WORDS, ['price','service','food','accomodation'] )

In [21]:
from itertools import combinations
# 하나의 토픽 내에서 nC2
# 큰 값이 좋은 것임
def cosine_sim(df):
    for col in df.columns:
        cos_val = 0
        words = df[col].tolist()
        for c in combinations(words,2):
#             print(c)
            cos_val += 1-cosine(word2glove[c[0]], word2glove[c[1]])
#             print(c)
#             print(cosine(word2glove[c[0]], word2glove[c[1]]))
        print(col, cos_val)
cosine_sim(DMM_LDA_Top_words)
print()
cosine_sim(DMM_CLDA_Top_words)

Topic 1 21.872375379888407
Topic 2 22.47562009205711
Topic 3 25.758627516870614
Topic 4 21.054852017700256

price 19.692479310674916
service 18.561523745732725
food 24.306535028088533
accomodation 17.68359948225043


In [22]:
# 토픽별로 cross similarity
# 작은 값이 좋은 것임
def cosine_sim_cross_topic(df):
    cols = list(df.columns)
    for col in cols:
        cos_val = 0
        target_words = df[col].tolist()
        rest_cols = list(set(cols) - set([col]))
        for not_col in rest_cols:
#             print(col, not_col)
            cross_words = df[not_col].tolist()
            for target_word in target_words:
                for cross_word in cross_words:
#                     print(target_word, cross_word)
                    cos_val += 1-cosine(word2glove[target_word],
                               word2glove[cross_word])
        print(col, cos_val)
cosine_sim_cross_topic(DMM_LDA_Top_words)
print()
cosine_sim_cross_topic(DMM_CLDA_Top_words)

Topic 1 160.65672745128927
Topic 2 158.89858022254487
Topic 3 166.41538963266825
Topic 4 157.73314261930278

price 113.85730698855456
service 107.91229207469563
food 102.85794865257414
accomodation 108.9873472334934


### word2vec 이용

In [23]:
import gensim

w2v_dir = '/Users/shinbo/Desktop/metting/LDA/paper/word_embedding/GoogleNews-vectors-negative300.bin/GoogleNews-vectors-negative300.bin'
word2vec_model = gensim.models.keyedvectors.KeyedVectors.load_word2vec_format(
    w2v_dir, binary=True
)

In [29]:
# 하나의 토픽 내에서 nC2
# 큰 값이 좋은 것임
def cosine_sim_w2v(df):
    total = 0
    for col in df.columns:
        cos_val = 0
        words = df[col].tolist()
        for c in combinations(words,2):
#             print(c)
            cos_val += 1-cosine(word2vec_model.get_vector(c[0]),
                               word2vec_model.get_vector(c[1]))
#             print(c)
#             print(cosine(word2glove[c[0]], word2glove[c[1]]))
        print(col, cos_val)
        total += cos_val
    print(total)
cosine_sim_w2v(DMM_LDA_Top_words)
print()
cosine_sim_w2v(DMM_CLDA_Top_words)

Topic 1 6.81068236194551
Topic 2 7.4617720246315
Topic 3 6.506621167063713
Topic 4 6.417552722617984
27.196628276258707

price 7.411576837301254
service 6.9898152351379395
food 16.137065947055817
accomodation 5.366100776940584
35.904558796435595


In [30]:
# 토픽별로 cross similarity
# 작은 값이 좋은 것임
def cosine_sim_w2v_cross_topic(df):
    total = 0
    cols = list(df.columns)
    for col in cols:
        cos_val = 0
        target_words = df[col].tolist()
        rest_cols = list(set(cols) - set([col]))
        for not_col in rest_cols:
#             print(col, not_col)
            cross_words = df[not_col].tolist()
            for target_word in target_words:
                for cross_word in cross_words:
#                     print(target_word, cross_word)
                    cos_val += 1-cosine(word2vec_model.get_vector(target_word),
                               word2vec_model.get_vector(cross_word))
        print(col, cos_val)
        total += cos_val
    print(total)
cosine_sim_w2v_cross_topic(DMM_LDA_Top_words)
print()
cosine_sim_w2v_cross_topic(DMM_CLDA_Top_words)

Topic 1 62.905876487493515
Topic 2 60.25979792140424
Topic 3 58.78958444669843
Topic 4 59.17897048778832
241.1342293433845

price 40.90699869324453
service 32.94442398485262
food 40.108993045054376
accomodation 36.398743920377456
150.35915964352898
